# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,295.89,81,100,10.55,MU,1728571039
1,1,ushuaia,-54.8000,-68.3000,279.96,65,75,10.29,AR,1728571041
2,2,ceelbuur,4.6850,46.6176,305.61,38,81,7.21,SO,1728571042
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,272.48,64,0,3.00,RU,1728571044
4,4,yellowknife,62.4560,-114.3525,273.94,100,75,2.06,CA,1728571046


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[city_data_df['Max Temp'] > 70]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
print(filtered_cities_df.head())

   City_ID                      City      Lat       Lng  Max Temp  Humidity  \
0        0             port mathurin -19.6833   63.4167    295.89        81   
1        1                   ushuaia -54.8000  -68.3000    279.96        65   
2        2                  ceelbuur   4.6850   46.6176    305.61        38   
3        3  petropavlovsk-kamchatsky  53.0452  158.6483    272.48        64   
4        4               yellowknife  62.4560 -114.3525    273.94       100   

   Cloudiness  Wind Speed Country        Date  
0         100       10.55      MU  1728571039  
1          75       10.29      AR  1728571041  
2          81        7.21      SO  1728571042  
3           0        3.00      RU  1728571044  
4          75        2.06      CA  1728571046  


In [19]:
# Count the number of cities in your DataFrame
num_cities = hotel_df['City'].nunique()
print(f"Number of unique cities in the DataFrame: {num_cities}")


Number of unique cities in the DataFrame: 583


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

                       City Country      Lat       Lng  Humidity Hotel Name
0             port mathurin      MU -19.6833   63.4167        81           
1                   ushuaia      AR -54.8000  -68.3000        65           
2                  ceelbuur      SO   4.6850   46.6176        38           
3  petropavlovsk-kamchatsky      RU  53.0452  158.6483        64           
4               yellowknife      CA  62.4560 -114.3525       100           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 1000  # Set the radius to 1000 meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
ceelbuur - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
yellowknife - nearest hotel: No hotel found
banjarmasin - nearest hotel: Hotel Grand Mentari
nar'yan-mar - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
vorkuta - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
ruyigi - nearest hotel: No hotel found
nemuro - nearest hotel: お宿エクハシ
nagqu - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
hamilton - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
aasiaat - nearest hotel: SØMA
waitangi - nearest hotel: Hotel Chathams
ribeira grande - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
albany - nearest hotel: No hotel found
port-aux-francais - nearest hotel: K

long beach - nearest hotel: Westin
smithers - nearest hotel: Sunshine Inn Hotel
niteroi - nearest hotel: No hotel found
suntar - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
kuala belait - nearest hotel: Hotel Sentosa
taunggyi - nearest hotel: Golden Island Cottages
meadow lake - nearest hotel: No hotel found
keflavik - nearest hotel: Núpan Deluxe
saint-philippe - nearest hotel: No hotel found
jaguey grande - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
fang - nearest hotel: Thipdararat Hotel
bhimunipatnam - nearest hotel: No hotel found
grand-popo - nearest hotel: Hotel Mon Secours
el haouaria - nearest hotel: Villa Zembra Maison d'hôtes
vinh - nearest hotel: Mường Thanh Luxury Song Lam Hotel
sinabang - nearest hotel: No hotel found
progreso - nearest hotel: Centro Vacacional Obrero CTM
kareli - nearest hotel: No hotel found
arkalyk - nearest hotel: Гостиница "Арсенал"
chinhoyi - nearest hotel: No 

sao felix do xingu - nearest hotel: Hotel Terraço
patong - nearest hotel: Thara Patong Beach Resort & Spa
mossel bay - nearest hotel: Protea Hotel Mossel Bay
cabo san lucas - nearest hotel: Comfort Rooms
upata - nearest hotel: Hotel Garibardi
edd - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
terra santa - nearest hotel: Hotel Novo Horizonte
kisangani - nearest hotel: Hotel Kisangani
maceio - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
puerto del rosario - nearest hotel: Hotel Tamasite
krasnovishersk - nearest hotel: Отель
balkashino - nearest hotel: No hotel found
brooks - nearest hotel: Canalta Hotel Brooks
kirkwall - nearest hotel: The Kirkwall Hotel
maamba - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
sao paulo de olivenca - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
uruzgan - nearest hotel: No hotel found
filadelfia - nearest hotel: The Ritz-Carlton, Philadelphia
yeppoon - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,81,Escale Vacances
1,ushuaia,AR,-54.8000,-68.3000,65,Apart Hotel Aires del Beagle
2,ceelbuur,SO,4.6850,46.6176,38,No hotel found
3,petropavlovsk-kamchatsky,RU,53.0452,158.6483,64,ООО Постоялый двор
4,yellowknife,CA,62.4560,-114.3525,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)